In [28]:
import pandas as pd
import numpy as np
import pickle, warnings, datetime
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [29]:
with open('../../Data/Processed/GDELT_Clean_202506141555.pkl', 'rb') as f:
    gdelt = pickle.load(f)

with open('../../Data/Processed/stock_data.pkl', 'rb') as f:
    stocks = pickle.load(f)

with open('../../Data/Processed/oil_data.pkl', 'rb') as f:
    oil = pickle.load(f)

In [30]:
gdelt

,PCA_GKG1_0_general,PCA_GKG1_1_general,PCA_GKG1_2_general,PCA_GKG1_3_general,PCA_GKG1_4_general,PCA_Scored_0_general,PCA_Scored_1_general,PCA_Scored_2_general,PCA_Scored_3_general,PCA_Scored_4_general,...,PCA_Word_1_Allegiant Air_rolling26,PCA_Word_1_Allegiant Air_rolling26_diff,PCA_Word_2_Allegiant Air_rolling26,PCA_Word_2_Allegiant Air_rolling26_diff,PCA_Word_3_Allegiant Air_rolling26,PCA_Word_3_Allegiant Air_rolling26_diff,PCA_Word_4_Allegiant Air_rolling26,PCA_Word_4_Allegiant Air_rolling26_diff,Article Count_Allegiant Air_rolling26,Article Count_Allegiant Air_rolling26_diff
stock_time,,,,,,,,,,,,,,,,,,,,,
2018-01-02 09:15:00,-33.720902,95.205623,87.552606,98.310646,-96.136658,78.906686,77.147308,74.122819,79.306570,77.280079,...,0.011835,0.500204,0.029393,0.499786,0.789585,0.500000,0.015975,0.500204,0.0048,0.500000
2018-01-02 09:30:00,-0.367777,0.539686,0.495139,0.533391,-0.535151,0.506796,0.509160,0.246953,0.510524,0.504305,...,0.011835,0.500204,0.029393,0.499786,0.789585,0.500000,0.015975,0.500204,0.0048,0.500000
2018-01-02 09:45:00,-0.181130,0.058680,0.129602,0.051890,-0.068351,0.168855,0.154376,0.362616,0.169844,0.174437,...,0.011835,0.500204,0.029393,0.499786,0.789585,0.500000,0.015975,0.500204,0.0048,0.500000
2018-01-02 10:00:00,0.164014,0.805351,0.650515,0.841479,-0.793925,0.533763,0.533249,0.206748,0.524511,0.471974,...,0.011835,0.500204,0.029393,0.499786,0.789585,0.500000,0.015975,0.500204,0.0048,0.500000
2018-01-02 10:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.011835,0.500204,0.029393,0.499786,0.789585,0.500000,0.015975,0.500204,0.0048,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30 14:45:00,-0.290557,0.209248,0.152521,0.196881,-0.190921,0.174081,0.170052,0.280851,0.160887,0.165472,...,0.002459,0.500204,0.019596,0.499786,0.802160,0.500000,0.004932,0.500204,0.0048,0.500000
2025-05-30 15:00:00,-1.207044,0.671244,0.955081,0.620670,-0.694152,1.000964,1.013993,0.981263,1.008438,0.999357,...,0.002459,0.500204,0.019596,0.499786,0.802160,0.500000,0.004932,0.500204,0.0048,0.500000
2025-05-30 15:15:00,-0.031969,0.475448,0.597537,0.455303,-0.495168,0.748410,0.711636,1.317726,0.672782,0.701768,...,0.002459,0.500204,0.019596,0.499786,0.802160,0.500000,0.004932,0.500204,0.0048,0.500000


In [31]:
stocks

,AAL_%Chg,ALGT_%Chg,ALK_%Chg,DAL_%Chg,ITA_%Chg,IYT_%Chg,JBLU_%Chg,JETS_%Chg,LUV_%Chg,UAL_%Chg,...,AAL_Volume,ALGT_Volume,ALK_Volume,DAL_Volume,ITA_Volume,IYT_Volume,JBLU_Volume,JETS_Volume,LUV_Volume,UAL_Volume
Time,,,,,,,,,,,,,,,,,,,,,
2018-01-02 09:30:00,0.13,0.52,1.31,0.31,0.08,0.80,0.13,0.68,0.49,0.43,...,181986.0,3789.0,47871.0,344480.0,28244.0,14312.0,293572.0,1448.0,224948.0,116039.0
2018-01-02 09:45:00,0.19,-0.88,0.05,-0.13,-0.12,0.07,-0.25,-0.04,0.43,-0.25,...,175661.0,1500.0,78358.0,263594.0,9632.0,7840.0,121302.0,302.0,208563.0,130163.0
2018-01-02 10:00:00,-0.08,1.09,0.18,0.22,0.17,0.23,0.27,0.32,-0.18,0.13,...,102270.0,7577.0,33292.0,219949.0,12906.0,14784.0,103446.0,500.0,135579.0,63010.0
2018-01-02 10:15:00,0.02,-0.03,0.20,0.03,-0.16,-0.29,-0.38,-0.01,-0.27,0.28,...,97181.0,7400.0,48838.0,122923.0,7462.0,8600.0,59784.0,300.0,87538.0,101656.0
2018-01-02 10:30:00,0.04,-0.74,0.05,0.30,-0.18,-0.09,-0.04,-0.08,0.30,0.24,...,112874.0,7972.0,44382.0,216575.0,11720.0,18240.0,56808.0,805.0,127150.0,63372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30 14:45:00,0.22,0.13,0.09,0.02,0.10,0.12,0.39,0.09,0.12,0.29,...,1445222.0,6516.0,27711.0,261687.0,9343.0,738.0,791081.0,19757.0,171924.0,144655.0
2025-05-30 15:00:00,0.09,-0.03,-0.09,0.21,0.11,0.02,-0.39,0.07,0.21,0.05,...,1866339.0,5159.0,43059.0,206491.0,9035.0,3369.0,763213.0,30811.0,228804.0,169821.0
2025-05-30 15:15:00,0.26,0.21,0.24,0.35,0.22,0.18,0.29,0.22,0.21,0.49,...,1436577.0,11567.0,70278.0,394754.0,12738.0,3297.0,1133607.0,64582.0,281941.0,402686.0


In [32]:
oil

,Oil_Open,Oil_High,Oil_Low,Oil_Last,Oil_Change,Oil_%Chg,Oil_Volume,Oil_High-Low,Oil_High-Low%,Oil_Last-Open,Oil_Last-Open%,Oil_Roll_SD_Last_10,Oil_Roll_SDSD_Last_10,Oil_Roll_SD_Volume_10,Oil_Roll_SDSD_Volume_10,Oil_MA4_Last,Oil_MA12_Last
Time,,,,,,,,,,,,,,,,,
2018-01-02 09:30:00,18.170,18.1800,18.100,18.1061,0.0061,0.03,11817.0,0.0800,0.000550,0.0639,0.003529,0.033187,0.003449,4931.431121,1044.505444,18.106525,18.129258
2018-01-02 09:45:00,18.110,18.1400,18.095,18.1000,-0.0061,-0.03,21330.0,0.0450,0.001657,0.0100,0.000552,0.031982,0.003440,6799.912909,1407.459022,18.106525,18.125925
2018-01-02 10:00:00,18.120,18.1419,18.120,18.1400,0.0400,0.22,21115.0,0.0219,0.001209,-0.0200,-0.001103,0.022526,0.003694,7922.457795,1792.904236,18.111525,18.125925
2018-01-02 10:15:00,18.140,18.1553,18.080,18.0800,-0.0600,-0.33,3478.0,0.0753,0.000843,0.0600,0.003319,0.015480,0.005434,8025.061814,1936.139595,18.106525,18.119258
2018-01-02 10:30:00,18.100,18.1000,18.070,18.0850,0.0050,0.03,9475.0,0.0300,0.000000,0.0150,0.000829,0.016914,0.006495,7987.519591,2046.916279,18.101250,18.110508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30 14:45:00,26.880,26.8850,26.870,26.8700,0.0100,0.04,8001.0,0.0150,0.000186,0.0100,0.000372,0.064313,0.009584,32786.197212,3654.157067,26.880000,26.820208
2025-05-30 15:00:00,26.870,26.8850,26.860,26.8600,-0.0100,-0.04,5610.0,0.0250,0.000558,0.0100,0.000372,0.045583,0.012929,32945.592401,4115.168148,26.870000,26.825208
2025-05-30 15:15:00,26.855,26.8800,26.825,26.8800,0.0200,0.07,22480.0,0.0550,0.000931,-0.0250,-0.000930,0.021986,0.019989,32478.332929,4596.320211,26.867500,26.844375


In [33]:
# Merge the datasets by row index
df = pd.merge(stocks, gdelt, left_index=True, right_index=True, how='inner')
df = pd.merge(df, oil, left_index=True, right_index=True, how='left')

In [34]:
df[df['Oil_High'].isna()]

,AAL_%Chg,ALGT_%Chg,ALK_%Chg,DAL_%Chg,ITA_%Chg,IYT_%Chg,JBLU_%Chg,JETS_%Chg,LUV_%Chg,UAL_%Chg,...,Oil_High-Low,Oil_High-Low%,Oil_Last-Open,Oil_Last-Open%,Oil_Roll_SD_Last_10,Oil_Roll_SDSD_Last_10,Oil_Roll_SD_Volume_10,Oil_Roll_SDSD_Volume_10,Oil_MA4_Last,Oil_MA12_Last
2018-01-09 14:45:00,0.02,-0.13,-0.07,0.01,-0.06,0.04,0.14,-0.06,-0.02,-0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-09 15:00:00,-0.10,-0.10,0.10,0.05,-0.02,0.07,0.14,-0.06,-0.26,-0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-19 10:15:00,0.15,-0.53,0.01,0.37,0.03,0.28,0.40,0.08,0.30,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-19 11:45:00,-0.16,0.06,-0.47,-0.07,-0.10,0.00,-0.18,0.20,-0.25,-0.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-19 12:15:00,-0.24,0.03,-0.33,-0.07,0.05,-0.01,-0.20,-0.23,-0.14,-0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-11 13:30:00,0.34,-0.01,0.34,0.15,0.21,-0.03,0.15,0.19,0.07,0.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-19 15:15:00,0.03,-0.36,-0.30,-0.25,-0.01,0.01,0.05,-0.14,0.01,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-20 13:15:00,-0.13,-0.20,0.11,-0.36,-0.06,-0.15,-0.42,-0.10,-0.08,-0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-20 13:30:00,0.29,0.52,0.17,0.29,0.15,0.22,0.63,0.28,0.17,0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
list(df.columns)

['AAL_%Chg',
 'ALGT_%Chg',
 'ALK_%Chg',
 'DAL_%Chg',
 'ITA_%Chg',
 'IYT_%Chg',
 'JBLU_%Chg',
 'JETS_%Chg',
 'LUV_%Chg',
 'UAL_%Chg',
 'AAL_Change',
 'ALGT_Change',
 'ALK_Change',
 'DAL_Change',
 'ITA_Change',
 'IYT_Change',
 'JBLU_Change',
 'JETS_Change',
 'LUV_Change',
 'UAL_Change',
 'AAL_High',
 'ALGT_High',
 'ALK_High',
 'DAL_High',
 'ITA_High',
 'IYT_High',
 'JBLU_High',
 'JETS_High',
 'LUV_High',
 'UAL_High',
 'AAL_High-Low',
 'ALGT_High-Low',
 'ALK_High-Low',
 'DAL_High-Low',
 'ITA_High-Low',
 'IYT_High-Low',
 'JBLU_High-Low',
 'JETS_High-Low',
 'LUV_High-Low',
 'UAL_High-Low',
 'AAL_High-Low%',
 'ALGT_High-Low%',
 'ALK_High-Low%',
 'DAL_High-Low%',
 'ITA_High-Low%',
 'IYT_High-Low%',
 'JBLU_High-Low%',
 'JETS_High-Low%',
 'LUV_High-Low%',
 'UAL_High-Low%',
 'AAL_Last',
 'ALGT_Last',
 'ALK_Last',
 'DAL_Last',
 'ITA_Last',
 'IYT_Last',
 'JBLU_Last',
 'JETS_Last',
 'LUV_Last',
 'UAL_Last',
 'AAL_Last-Open',
 'ALGT_Last-Open',
 'ALK_Last-Open',
 'DAL_Last-Open',
 'ITA_Last-Open',
 

In [36]:
# Ensure no missing values
df = df.ffill().bfill()

In [37]:
# Renaming GDELT columns from airline name to stock ticker
df.columns = [col.replace('Allegiant Air', 'ALGT') for col in df.columns]
df.columns = [col.replace('Alaska Airlines', 'ALK') for col in df.columns]
df.columns = [col.replace('United Airlines', 'UAL') for col in df.columns]
df.columns = [col.replace('Delta Air Lines', 'DAL') for col in df.columns]
df.columns = [col.replace('JetBlue', 'JBLU') for col in df.columns]
df.columns = [col.replace('Southwest Airlines', 'LUV') for col in df.columns]
df.columns = [col.replace('American Airlines', 'AAL') for col in df.columns]

In [38]:
# Create day of week dummies
df['day_of_week']   = df.index.dayofweek
df['month_of_year'] = df.index.month
df['hour_of_day']   = df.index.hour

# Create dummy variables for categorical features
df = pd.get_dummies(df, columns=['day_of_week', 'month_of_year', 'hour_of_day'], drop_first=False)

# Create a column which is 1 when time is 9:30am, and 0 otherwise
df['is_open'] = np.where((df.index.hour == 9) & (df.index.minute == 30), 1, 0)

In [39]:
df

,AAL_%Chg,ALGT_%Chg,ALK_%Chg,DAL_%Chg,ITA_%Chg,IYT_%Chg,JBLU_%Chg,JETS_%Chg,LUV_%Chg,UAL_%Chg,...,month_of_year_11,month_of_year_12,hour_of_day_9,hour_of_day_10,hour_of_day_11,hour_of_day_12,hour_of_day_13,hour_of_day_14,hour_of_day_15,is_open
2018-01-02 09:30:00,0.13,0.52,1.31,0.31,0.08,0.80,0.13,0.68,0.49,0.43,...,False,False,True,False,False,False,False,False,False,1
2018-01-02 09:45:00,0.19,-0.88,0.05,-0.13,-0.12,0.07,-0.25,-0.04,0.43,-0.25,...,False,False,True,False,False,False,False,False,False,0
2018-01-02 10:00:00,-0.08,1.09,0.18,0.22,0.17,0.23,0.27,0.32,-0.18,0.13,...,False,False,False,True,False,False,False,False,False,0
2018-01-02 10:15:00,0.02,-0.03,0.20,0.03,-0.16,-0.29,-0.38,-0.01,-0.27,0.28,...,False,False,False,True,False,False,False,False,False,0
2018-01-02 10:30:00,0.04,-0.74,0.05,0.30,-0.18,-0.09,-0.04,-0.08,0.30,0.24,...,False,False,False,True,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-30 14:45:00,0.22,0.13,0.09,0.02,0.10,0.12,0.39,0.09,0.12,0.29,...,False,False,False,False,False,False,False,True,False,0
2025-05-30 15:00:00,0.09,-0.03,-0.09,0.21,0.11,0.02,-0.39,0.07,0.21,0.05,...,False,False,False,False,False,False,False,False,True,0
2025-05-30 15:15:00,0.26,0.21,0.24,0.35,0.22,0.18,0.29,0.22,0.21,0.49,...,False,False,False,False,False,False,False,False,True,0
2025-05-30 15:30:00,-0.13,-0.11,-0.31,-0.24,-0.11,-0.14,-0.49,-0.09,-0.15,-0.19,...,False,False,False,False,False,False,False,False,True,0


In [40]:
# Write to pickle
with open('../../Data/Processed/merged_data_202506141602.pkl', 'wb') as f:
    pickle.dump(df, f)